In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from functools import reduce
from pyspark.sql import DataFrame
from pyspark.sql.types import StringType,DoubleType,FloatType
from pyspark.sql.functions import udf
from pyspark.sql.functions import regexp_replace
from pyspark.sql.functions import lit
import locale
import sys
import os
import re

In [2]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [3]:
spark

# Leyendo lista de Colegios y Estudiantes

In [4]:
base = '../Colegios'
base2 = '../SB'
colegios = os.listdir('../Colegios')
estudiantes = os.listdir('../SB')

In [5]:
estudiantes.sort()
colegios.sort()
estuDic = {}
coleDic = {}
for estu in estudiantes:
    estuDic[re.findall('\d+', estu)[1]] = estu

for colegio in colegios:
    coleDic[re.findall('\d+', colegio)[1]] = colegio

In [6]:
# Funciòn para unir cualquier nùmero de dataframes con la misma estructura 
def unionAll(*dfs):
    return reduce(DataFrame.unionAll, dfs)

In [7]:
print("Archivos de estudiantes: ", estuDic['20101'],estuDic['20102'])
print("Archivos de colegios: ",coleDic['2010'])

Archivos de estudiantes:  SB11_20101.txt SB11_20102.txt
Archivos de colegios:  SB11-CLASIFI-PLANTELES-2010.txt


In [8]:
columnasPuntos20101 = [
 'cole_cod_dane_institucion',
 'punt_lenguaje',
 'punt_matematicas',
 'punt_c_sociales',
 'punt_filosofia',
 'punt_biologia',
 'punt_quimica',
 'punt_fisica',
 'punt_ingles',
 'punt_profundizacion',
 'punt_interdisciplinar']

columnasPuntos20102 = [
 'cole_cod_dane_institucion',
 'punt_lenguaje',
 'punt_matematicas',
 'punt_c_sociales',
 'punt_filosofia',
 'punt_biologia',
 'punt_quimica',
 'punt_fisica',
 'punt_ingles',
 'punt_comp_flexible']

# Columnas en comun entre el primero y el segundo semestre
columnasPuntos = [
 'cole_cod_dane_institucion',
 'punt_lenguaje',
 'punt_matematicas',
 'punt_c_sociales',
 'punt_filosofia',
 'punt_biologia',
 'punt_quimica',
 'punt_fisica',
 'punt_ingles']


# https://sparkbyexamples.com/spark/rename-a-column-on-spark-dataframes/

In [9]:
# from pyspark.sql.functions import lit
# df = df.withColumn('año',lit(2009))
#df1 = spark.read.load(base2 + '/' + estuDic['20101'],"com.databricks.spark.csv",header='true',inferSchema='true',sep='¬').select(columnasPuntos20101)
#columnasPuntos20101.remove('cole_cod_dane_institucion')
#for cp in columnasPuntos20101:
#    df1 = df1.withColumn(cp, regexp_replace(cp, ',', '.'))
#    df1 = df1.withColumn(cp, df[cp].cast("float"))

In [10]:
# SE leen los dos archivos y se crean dos dataframes
df1 = spark.read.load(base2 + '/' + estuDic['20101'],"com.databricks.spark.csv",header='true',inferSchema='true',sep='¬').select(columnasPuntos20101).withColumn('punt_comp_flexible',lit(0))
df2 = spark.read.load(base2 + '/' + estuDic['20102'],"com.databricks.spark.csv",header='true',inferSchema='true',sep='¬').select(columnasPuntos20102).withColumn('punt_profundizacion',lit(0)).withColumn('punt_interdisciplinar',lit(0))

In [11]:
#dft = df1.columns.remove('cole_cod_dane_institucion')
cols = df1.columns
cols.remove('cole_cod_dane_institucion')
cols

['punt_lenguaje',
 'punt_matematicas',
 'punt_c_sociales',
 'punt_filosofia',
 'punt_biologia',
 'punt_quimica',
 'punt_fisica',
 'punt_ingles',
 'punt_profundizacion',
 'punt_interdisciplinar',
 'punt_comp_flexible']

In [12]:
# SE unen los dos dataframes
df = unionAll(df1,df2)
for cp in cols:
    df = df.withColumn(cp, regexp_replace(cp, ',', '.'))
    df = df.withColumn(cp, df[cp].cast("float"))
    
df1 = df2 = None

In [13]:
print("Numero de filas ", df.count())

Numero de filas  570844


In [14]:
print(df.printSchema())

root
 |-- cole_cod_dane_institucion: string (nullable = true)
 |-- punt_lenguaje: float (nullable = true)
 |-- punt_matematicas: float (nullable = true)
 |-- punt_c_sociales: float (nullable = true)
 |-- punt_filosofia: float (nullable = true)
 |-- punt_biologia: float (nullable = true)
 |-- punt_quimica: float (nullable = true)
 |-- punt_fisica: float (nullable = true)
 |-- punt_ingles: float (nullable = true)
 |-- punt_profundizacion: float (nullable = true)
 |-- punt_interdisciplinar: float (nullable = true)
 |-- punt_comp_flexible: float (nullable = true)

None


In [15]:
# Contando el numero de estudiantes por instituciòn y mostrandola de forma descendente
dsCountEst = df.groupBy('cole_cod_dane_institucion').count().orderBy('count',ascending=
False)

In [16]:
dsCountEst.show(10,False)

+-------------------------+-----+
|cole_cod_dane_institucion|count|
+-------------------------+-----+
|null                     |5572 |
|105001000108             |1205 |
|311001105391             |904  |
|325754005072             |904  |
|354001001520             |849  |
|105001013340             |845  |
|305615001312             |802  |
|305266019802             |715  |
|368001005488             |686  |
|168001003591             |681  |
+-------------------------+-----+
only showing top 10 rows



In [17]:
#Se crea una columna con el total de los puntajes   
df= df.withColumn("Suma",df['punt_lenguaje']+df['punt_matematicas']+df['punt_c_sociales']+
                     df['punt_filosofia']+df['punt_biologia']+df['punt_quimica']+df['punt_fisica']+
                     df['punt_ingles']+df['punt_profundizacion']+df['punt_interdisciplinar']+df['punt_comp_flexible'])

## Resultados finales por instituciòn

In [18]:
# Còdigo para obtener la media de los resultados de las evaluaciones de todos los estudiantes por instituciòn 
# y posiciòn en orden descendente.
from pyspark.sql.functions import *
from pyspark.sql.window import Window

df = df.groupBy('cole_cod_dane_institucion').mean()
df = df.withColumn("rank", dense_rank().over(Window.orderBy(desc("avg(Suma)"))))
df = df.orderBy('rank',ascending= True)

In [19]:
df.select('cole_cod_dane_institucion','avg(Suma)','rank').show(10,False)

+-------------------------+-----------------+----+
|cole_cod_dane_institucion|avg(Suma)        |rank|
+-------------------------+-----------------+----+
|311001089221             |668.7444424099392|1   |
|308001005299             |647.999994187128 |2   |
|311001065489             |631.8419738769531|3   |
|311848000308             |627.9298460092117|4   |
|305001006831             |626.3600158691406|5   |
|468547002051             |618.5052992876838|6   |
|373001007662             |617.6485682896206|7   |
|368001000729             |615.6911320880176|8   |
|311001086320             |613.2305054483535|9   |
|311001076634             |605.8716648646763|10  |
+-------------------------+-----------------+----+
only showing top 10 rows



## Colegios

In [21]:
# SE crea Dataframe de colegios en este año desaparecio la columna COLE_CODMPIO_COLEGIO
dfc = spark.read.load(base + '/' + coleDic['2010'],"com.databricks.spark.csv",header='true',inferSchema='true',sep='\t')
dfc = dfc.withColumn('COLE_CODIGO_COLEGIO', dfc['COLE_CODIGO_COLEGIO'].cast("string"))

In [22]:
print(dfc.printSchema())

root
 |-- COLE_CODIGO_COLEGIO: string (nullable = true)
 |-- COLE_INST_NOMBRE: string (nullable = true)
 |-- COLE_COD_CIUDAD: integer (nullable = true)
 |-- COLE_MPIO_COLEGIO: string (nullable = true)
 |-- COLE_DEPTO_COLEGIO: string (nullable = true)
 |-- COLE_INST_JORNADA: string (nullable = true)
 |-- COLE_CALENDARIO_COLEGIO: string (nullable = true)
 |-- COLE_GENEROPOBLACION: string (nullable = true)
 |-- COLE_NATURALEZA: string (nullable = true)
 |-- COLE_CIENCIAS_SOCIALES: integer (nullable = true)
 |-- COLE_QUIMICA: integer (nullable = true)
 |-- COLE_FISICA: integer (nullable = true)
 |-- COLE_BIOLOGIA: integer (nullable = true)
 |-- COLE_FILOSOFIA: integer (nullable = true)
 |-- COLE_MATEMATICAS: integer (nullable = true)
 |-- COLE_LENGUAJE: integer (nullable = true)
 |-- COLE_INGLES: integer (nullable = true)
 |-- COLE_GEOGRAFIA: string (nullable = true)
 |-- COLE_HISTORIA: string (nullable = true)
 |-- COLE_CATEGORIA: string (nullable = true)
 |-- COLE_ESTUDIANTES_PRESENTES: 

In [23]:
colsColegio = ['COLE_CODIGO_COLEGIO',
 'COLE_CIENCIAS_SOCIALES',
 'COLE_QUIMICA',
 'COLE_FISICA',
 'COLE_BIOLOGIA',
 'COLE_FILOSOFIA',
 'COLE_MATEMATICAS',
 'COLE_LENGUAJE',
 'COLE_INGLES','COLE_GEOGRAFIA','COLE_HISTORIA']

In [24]:
dfc = dfc.select(colsColegio)

In [25]:
dfc.show(5,False)

+-------------------+----------------------+------------+-----------+-------------+--------------+----------------+-------------+-----------+--------------+-------------+
|COLE_CODIGO_COLEGIO|COLE_CIENCIAS_SOCIALES|COLE_QUIMICA|COLE_FISICA|COLE_BIOLOGIA|COLE_FILOSOFIA|COLE_MATEMATICAS|COLE_LENGUAJE|COLE_INGLES|COLE_GEOGRAFIA|COLE_HISTORIA|
+-------------------+----------------------+------------+-----------+-------------+--------------+----------------+-------------+-----------+--------------+-------------+
|43364              |7                     |6           |7          |6            |7             |6               |6            |6          |null          |null         |
|43372              |8                     |7           |7          |6            |9             |7               |7            |7          |null          |null         |
|43406              |7                     |6           |7          |6            |6             |6               |5            |6          |null

In [26]:
print("El numero de colegios es: ", dfc.select('COLE_CODIGO_COLEGIO').count())

El numero de colegios es:  10974


## Anàlisis comparativo entre estudiantes y colegios

In [27]:
dfjoin = df.join(dfc, df.cole_cod_dane_institucion == dfc.COLE_CODIGO_COLEGIO)

In [28]:
print("Nùmero de colegios coincidentes entre el archivo de estudiantes y colegios:", dfjoin.count())

Nùmero de colegios coincidentes entre el archivo de estudiantes y colegios: 0


La informaciòn de colegios no coincide con la informaciòn de estudiantes, no existe cole_cod_dane_institucion en el archivo de colegios.